In [ ]:
import torch
import torch.nn as nn

In [ ]:
def get_params(vocab_size, num_hiddens):
    # Input gate params
    W_hi = torch.normal(0, 0.01, (vocab_size, num_hiddens), requires_grad=True)
    W_xi = torch.normal(0, 0.01, (num_hiddens, num_hiddens), rewuires_grad=True)
    b_i = torch.zeros(num_hiddens, requires_grad=True)
    # Forget gate params
    W_hf = torch.normal(0, 0.01, (vocab_size, num_hiddens), requires_grad=True)
    W_xf = torch.normal(0, 0.01, (num_hiddens, num_hiddens), requires_grad=True)
    b_f = torch.zeros(num_hiddens, requires_grad=True)
    # Output gate params
    W_ho = torch.normal(0, 0.01, (vocab_size, num_hiddens), requires_grad=True)
    W_xo = torch.normal(0, 0.01, (num_hiddens, num_hiddens), requires_grad=True)
    b_o = torch.zeros(num_hiddens, requires_grad=True)
    # Memory cell params
    W_hc = torch.normal(0, 0.01, (vocab_size, num_hiddens), requires_grad=True)
    W_xc = torch.normal(0, 0.01, (num_hiddens, num_hiddens), requires_grad=True)
    b_c = torch.zeros(num_hiddens, requires_grad=True)
    # Output later params
    W_hq = torch.normal(0, 0.01, (num_hiddens, num_outputs), requires_grad=True)
    b_q = torch.zeros(num_outputs, requires_grad=True)
    params = [W_hi, W_xi, bi, W_hf, W_xf, b_f, W_ho, W_xo, b_o, W_hc, W_xc, b_c]
    return params

In [ ]:
def init_state(batch_size, num_hiddens):
    return (torch.zeros(batch_size, num_hiddens), torch.zeros(batch_size, num_hiddens))

In [ ]:
def lstm(inputs, state, params):
    W_hi, W_xi, bi, W_hf, W_xf, b_f, W_ho, W_xo, b_o, W_hc, W_xc, b_c = params
    H, C = init_state()
    outputs = []
    for x in inputs:
        # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=nn+lstm#torch.nn.LSTM
        I = torch.sigmoid((W_xi @ x) + (W_hi @ H) + b_i)
        F = torch.sigmoid((W_xf @ x) + (W_hf @ H) + b_f)
        O = torch.sigmoid((W_xo @ x) + (W_ho @ H) + b_o)
        G = torch.tanh((W_xc @ x) + (W_hc @ H) + b_c)
        C = F * C + I * G
        H = O * torch.tanh(C)
        Y = (W_hq @ H) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H, C)